### 상대모멘텀 
- 10개의 주식 데이터를 이용하여 투자 전략 테스팅
1. 월초부터 월말까지의 수정주가를 이용하여 월별 수익율
2. 월별 수익율이 높은 순서중 n개의 주식을 선택 
3. 해당하는 주식들을 매수 후 매도하여 수익율 계산

In [ ]:
import pandas as pd 
from datetime import datetime
import numpy as np
import os 
import glob
import warnings

In [ ]:
# warning message 제거 
warnings.filterwarnings('ignore')

In [110]:
# 월별 수익율을 계산하는 함수 생성 
def create_1M_rtn(_df, _ticker, _start = "2010-01-01", _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # 컬럼에 Date가 포함되어있는지 확인 
    if 'Date' in result.columns:
        result = result.loc[result['Date'] >= _start, ['Date', _col]]
        # Date 컬럼의 데이터를 시계열로 변경
        result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d')
        result.set_index('Date', inplace=True)
    else:
        result.index = pd.to_datetime(result.index, inplace=True)
        result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준 년월의 중복데이터를 제거하고 고유한 값들을 리스트로 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [111]:
aapl = pd.read_csv('../../csv/AAPL.csv')

In [112]:
sample_aapl, ym_list = create_1M_rtn(aapl, 'AAPL')

In [113]:
sample_aapl.head()

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL


In [114]:
ym_list

array(['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06',
       '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12',
       '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', 

In [115]:
# 특정 경로에 있는 파일들의 목록을 로드 
os.listdir("./data")

files = glob.glob("./data/*.csv")
files

['./data\\AAPL.csv',
 './data\\AMZN.csv',
 './data\\BND.csv',
 './data\\GDX.csv',
 './data\\GLD.csv',
 './data\\GM.csv',
 './data\\MSFT.csv',
 './data\\SLV.csv',
 './data\\SPY.csv',
 './data\\USM.csv']

In [116]:

# 새로운 데이터프레임을 생성 
# 종목별 데이터프레임 
stock_df = pd.DataFrame()
# 월말 데이터프레임 
month_last_df = pd.DataFrame()

for file in files:
    folder, name = os.path.split(file)
    # print(folder, name)
    head, tail = os.path.splitext(name)
    # print(head, tail)
    # head는 create_1M_rtn 함수에 ticker 인자값으로 사용

    # 데이터 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1M_rtn 함수를 호출 
    price_df, ym_list = create_1M_rtn(read_df, head)

    # 유니언 결합 (단순한 행 결합 함수)
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 월별 상태 모멘텀을 계산하기 위해 1개월간의 수익율 계산
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        m_rtn = price_df.loc[flag,].iloc[-1, 0] / price_df.loc[flag,].iloc[0, 0]
        price_df.loc[flag, '1m_rtn'] = m_rtn
        data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df, data], axis=0)

In [118]:
month_last_df.tail()

,CODE,1m_rtn
Date,,
2019-02-28,USM,0.817688
2019-03-29,USM,0.974942
2019-04-30,USM,1.035968
2019-05-31,USM,0.894067
2019-06-24,USM,1.087699


In [119]:
month_rtn_df = month_last_df.copy()

In [120]:
month_rtn_df.reset_index(inplace=True)

In [121]:
month_rtn_df.head()

,Date,CODE,1m_rtn
0,2010-01-29,AAPL,0.897435
1,2010-02-26,AAPL,1.050789
2,2010-03-31,AAPL,1.124456
3,2010-04-30,AAPL,1.106454
4,2010-05-28,AAPL,0.964445


In [122]:
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= '1m_rtn'
)

In [123]:
month_rtn_df.head()

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.897435,0.936595,1.011692,0.853490,0.965027,NaN,0.910501,0.922809,0.947586,0.870922
2010-02-26,1.050789,0.996046,1.003909,1.022124,1.009968,NaN,1.013871,0.982274,1.015404,1.004938
2010-03-31,1.124456,1.090172,0.996728,0.994848,0.995614,NaN,1.009304,1.063935,1.049976,1.117472
2010-04-30,1.106454,1.040134,1.011023,1.088577,1.046254,NaN,1.047325,1.041049,1.008574,0.996684
2010-05-28,0.964445,0.912503,1.011821,0.999599,1.027218,NaN,0.839806,0.978827,0.908766,0.968954


In [127]:
month_rtn_df = month_rtn_df.rank(
    axis=1, 
    ascending=False, 
    method='max', 
    pct=True
)

In [128]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.777778,0.444444,0.111111,1.000000,0.222222,NaN,0.666667,0.555556,0.333333,0.888889
2010-02-26,0.111111,0.888889,0.777778,0.222222,0.555556,NaN,0.444444,1.000000,0.333333,0.666667
2010-03-31,0.111111,0.333333,0.777778,1.000000,0.888889,NaN,0.666667,0.444444,0.555556,0.222222
2010-04-30,0.111111,0.666667,0.777778,0.222222,0.444444,NaN,0.333333,0.555556,0.888889,1.000000
2010-05-28,0.666667,0.777778,0.222222,0.333333,0.111111,NaN,1.000000,0.444444,0.888889,0.555556
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.200000,0.500000,0.600000,0.800000,0.700000,0.4,0.100000,0.900000,0.300000,1.000000
2019-03-29,0.100000,0.200000,0.500000,0.400000,0.700000,1.0,0.300000,0.800000,0.600000,0.900000
2019-04-30,0.300000,0.200000,0.700000,1.000000,0.800000,0.5,0.100000,0.900000,0.600000,0.400000


In [129]:
# 상위 15% 종목만 선택 
# where(조건식, 거짓일때 변경될 데이터)
month_rtn_df =  month_rtn_df.where(month_rtn_df < 0.15, 0)

In [130]:
month_rtn_df.head()

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.000000,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2010-02-26,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2010-03-31,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2010-04-30,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2010-05-28,0.000000,0.0,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0,0.0


In [132]:
# 데이터 중에서 0이 아닌 데이터는 1로 변경 
month_rtn_df[month_rtn_df != 0] = 1

In [133]:
month_rtn_df

CODE,AAPL,AMZN,BND,GDX,GLD,GM,MSFT,SLV,SPY,USM
Date,,,,,,,,,,
2010-01-29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-02-26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-03-31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-04-30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-05-28,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2019-02-28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-03-29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-04-30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [134]:
# CODE 값들을 유니크 데이터만 따로 출력 
stock_codes = stock_df['CODE'].unique()
stock_codes

array(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY',
       'USM'], dtype=object)

In [135]:
month_rtn_df.loc["2010-01-29 00:00:00", month_rtn_df.loc["2010-01-29 00:00:00"] >= 1].index

Index(['BND'], dtype='object', name='CODE')

In [136]:
sig_dict = dict()

for date in month_rtn_df.index:
    # print(date)
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] >= 1].index)
    # print(ticker_list)
    sig_dict[date] = ticker_list

In [137]:
sig_dict

{Timestamp('2010-01-29 00:00:00'): ['BND'],
 Timestamp('2010-02-26 00:00:00'): ['AAPL'],
 Timestamp('2010-03-31 00:00:00'): ['AAPL'],
 Timestamp('2010-04-30 00:00:00'): ['AAPL'],
 Timestamp('2010-05-28 00:00:00'): ['GLD'],
 Timestamp('2010-06-30 00:00:00'): ['GDX'],
 Timestamp('2010-07-30 00:00:00'): ['USM'],
 Timestamp('2010-08-31 00:00:00'): ['GDX'],
 Timestamp('2010-09-30 00:00:00'): ['AMZN'],
 Timestamp('2010-10-29 00:00:00'): ['SLV'],
 Timestamp('2010-11-30 00:00:00'): ['SLV'],
 Timestamp('2010-12-31 00:00:00'): ['SLV'],
 Timestamp('2011-01-31 00:00:00'): ['AAPL'],
 Timestamp('2011-02-28 00:00:00'): ['SLV'],
 Timestamp('2011-03-31 00:00:00'): ['SLV'],
 Timestamp('2011-04-29 00:00:00'): ['SLV'],
 Timestamp('2011-05-31 00:00:00'): ['BND'],
 Timestamp('2011-06-30 00:00:00'): ['MSFT'],
 Timestamp('2011-07-29 00:00:00'): ['SLV'],
 Timestamp('2011-08-31 00:00:00'): ['GLD'],
 Timestamp('2011-09-30 00:00:00'): ['AMZN'],
 Timestamp('2011-10-31 00:00:00'): ['GM'],
 Timestamp('2011-11-30 00:

In [138]:
# 거래 내역컬럼을 생성하는 데이터프레임 생성하는 함수 
def create_trade_book(_df, _code):
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book['p'+c] = ""
        book['r'+c] = ""
    return book

In [139]:
stock_df.head()

,Adj Close,STD-YM,1m_rtn,CODE
Date,,,,
2010-01-04,26.782711,2010-01,0,AAPL
2010-01-05,26.829010,2010-01,0,AAPL
2010-01-06,26.402260,2010-01,0,AAPL
2010-01-07,26.353460,2010-01,0,AAPL
2010-01-08,26.528664,2010-01,0,AAPL


In [140]:
stock_c_matrix = stock_df.reset_index().pivot_table(
    index='Date', 
    columns = 'CODE', 
    values= stock_df.columns[0]
)

In [ ]:
stock_c_matrix

In [142]:
book = create_trade_book(stock_c_matrix, stock_codes)

In [143]:
book.columns

Index(['AAPL', 'AMZN', 'BND', 'GDX', 'GLD', 'GM', 'MSFT', 'SLV', 'SPY', 'USM',
       'STD-YM', 'pAAPL', 'rAAPL', 'pAMZN', 'rAMZN', 'pBND', 'rBND', 'pGDX',
       'rGDX', 'pGLD', 'rGLD', 'pGM', 'rGM', 'pMSFT', 'rMSFT', 'pSLV', 'rSLV',
       'pSPY', 'rSPY', 'pUSM', 'rUSM'],
      dtype='object', name='CODE')

In [144]:
sig_dict.items()

dict_items([(Timestamp('2010-01-29 00:00:00'), ['BND']), (Timestamp('2010-02-26 00:00:00'), ['AAPL']), (Timestamp('2010-03-31 00:00:00'), ['AAPL']), (Timestamp('2010-04-30 00:00:00'), ['AAPL']), (Timestamp('2010-05-28 00:00:00'), ['GLD']), (Timestamp('2010-06-30 00:00:00'), ['GDX']), (Timestamp('2010-07-30 00:00:00'), ['USM']), (Timestamp('2010-08-31 00:00:00'), ['GDX']), (Timestamp('2010-09-30 00:00:00'), ['AMZN']), (Timestamp('2010-10-29 00:00:00'), ['SLV']), (Timestamp('2010-11-30 00:00:00'), ['SLV']), (Timestamp('2010-12-31 00:00:00'), ['SLV']), (Timestamp('2011-01-31 00:00:00'), ['AAPL']), (Timestamp('2011-02-28 00:00:00'), ['SLV']), (Timestamp('2011-03-31 00:00:00'), ['SLV']), (Timestamp('2011-04-29 00:00:00'), ['SLV']), (Timestamp('2011-05-31 00:00:00'), ['BND']), (Timestamp('2011-06-30 00:00:00'), ['MSFT']), (Timestamp('2011-07-29 00:00:00'), ['SLV']), (Timestamp('2011-08-31 00:00:00'), ['GLD']), (Timestamp('2011-09-30 00:00:00'), ['AMZN']), (Timestamp('2011-10-31 00:00:00'), [

In [145]:
# 포지션을 생성 
for date, values in sig_dict.items():
    # print(date, values)
    for stock in values:
        book.loc[date, 'p'+stock] = 'ready'+stock

In [146]:
book[['pBND']].iloc[15:25]

CODE,pBND
Date,
2010-01-26,
2010-01-27,
2010-01-28,
2010-01-29,readyBND
2010-02-01,
2010-02-02,
2010-02-03,
2010-02-04,
2010-02-05,


In [147]:
# 거래 내역 추가 
def trading(_book, s_code):
    std_ym = ""
    buy_phase = False

    # 종목별로 순회
    for code in s_code:
        for i in _book.index:
            # 해당 종목코드의 포지션을 잡는다.
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.shift().loc[i, 'p'+code] == "ready"+code):
                std_ym = book.loc[i, 'STD-YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있다면 매수 상태 유지 
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.loc[i, 'STD-YM'] == std_ym) & \
                (buy_phase):
                _book.loc[i, 'p'+code] = 'buy'+code
            
            # std_ym, buy_phase 초기화
            if _book.loc[i, 'p'+code] == "":
                std_ym = ""
                buy_phase = False
    return _book
                

In [148]:
book = trading(book, stock_codes)

In [149]:
book['pAAPL'].value_counts()

pAAPL
             2078
buyAAPL       292
readyAAPL      14
Name: count, dtype: int64

In [150]:
# 수익율 계산 함수 
def multi_returns(_book, s_code):
    rtn = 1
    buy_dict = dict()
    sell_dict = dict()

    for i in _book.index:
        for code in s_code:
            # 매수 (p+code 2일전에 "" 1일전에 ready 오늘이 buy)
            if (_book.shift(2).loc[i, "p"+code] == "") & \
                (_book.shift(1).loc[i, 'p'+code] == "ready"+code) & \
                (_book.loc[i, 'p'+code] == "buy"+code):
                buy_dict[code] = book.loc[i, code]
                print(f"매수일 : {i}, 종목코드 : {code}, 매수가 : {buy_dict[code]}")
            # 매도 (1일 전의 pcode가 buy 오늘의 pcode가 "")
            elif (_book.shift(1).loc[i, "p"+code] == "buy"+code) & \
                (_book.loc[i, 'p'+code] == ""):
                sell_dict[code] = _book.loc[i, code]
                # 수익율 계산 
                rtn = sell_dict[code] / buy_dict[code]
                _book.loc[i, 'r'+code] = rtn
                print(f"매도일 : {i}, 종목코드 : {code}, 매도가 : {sell_dict[code]}, 수익율 : {rtn}")
            # buy_dict, sell_dict 데이터를 초기화
            if _book.loc[i, 'p'+code] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
    return _book


In [151]:
rtn_book = multi_returns(book, stock_codes)

매수일 : 2010-02-01 00:00:00, 종목코드 : BND, 매수가 : 61.280487
매수일 : 2010-03-01 00:00:00, 종목코드 : AAPL, 매수가 : 26.154476
매도일 : 2010-03-01 00:00:00, 종목코드 : BND, 매도가 : 61.585163, 수익율 : 1.0049718273289832
매도일 : 2010-06-01 00:00:00, 종목코드 : AAPL, 매도가 : 32.642082, 수익율 : 1.248049549912604
매수일 : 2010-06-01 00:00:00, 종목코드 : GLD, 매수가 : 119.910004
매수일 : 2010-07-01 00:00:00, 종목코드 : GDX, 매수가 : 46.706642
매도일 : 2010-07-01 00:00:00, 종목코드 : GLD, 매도가 : 117.040001, 수익율 : 0.976065358149767
매도일 : 2010-08-02 00:00:00, 종목코드 : GDX, 매도가 : 45.228813, 수익율 : 0.9683593395560315
매수일 : 2010-08-02 00:00:00, 종목코드 : USM, 매수가 : 40.492416
매수일 : 2010-09-01 00:00:00, 종목코드 : GDX, 매수가 : 49.746994
매도일 : 2010-09-01 00:00:00, 종목코드 : USM, 매도가 : 36.667038, 수익율 : 0.9055285315650219
매수일 : 2010-10-01 00:00:00, 종목코드 : AMZN, 매수가 : 153.710007
매도일 : 2010-10-01 00:00:00, 종목코드 : GDX, 매도가 : 53.276817, 수익율 : 1.0709555033616704
매도일 : 2010-11-01 00:00:00, 종목코드 : AMZN, 매도가 : 162.580002, 수익율 : 1.0577060347150984
매수일 : 2010-11-01 00:00:00, 종목코드 : SLV, 매수가

In [152]:
# 누적 수익율 계산 함수 
def multi_acc_returns(_book, s_code):
    # 누적 수익율 변수 생성
    acc_rtn = 1
    for i in _book.index:
        count = 0
        rtn = 0
        for code in s_code:
            # 수익율 데이터가 존재하는 경우
            if _book.loc[i, 'r'+code]:
                count += 1
                rtn += _book.loc[i, "r"+code]
        if (rtn != 0) & (count != 0):
            acc_rtn *= (rtn / count)
            print(f"누적 매도일 : {i}, 매도 종목수 : {count}, 수익율 : {round(rtn/count, 2)}")
        _book.loc[i, 'acc_rtn'] = acc_rtn
    
    return _book, acc_rtn

In [153]:
multi_acc_returns(rtn_book, stock_codes)

누적 매도일 : 2010-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2010-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.25
누적 매도일 : 2010-07-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.98
누적 매도일 : 2010-08-02 00:00:00, 매도 종목수 : 1, 수익율 : 0.97
누적 매도일 : 2010-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.91
누적 매도일 : 2010-10-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.07
누적 매도일 : 2010-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-02-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.16
누적 매도일 : 2011-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.01
누적 매도일 : 2011-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-07-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.99
누적 매도일 : 2011-08-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.05
누적 매도일 : 2011-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-10-03 00:00:00, 매도 종목수 : 1, 수익율 : 0.9
누적 매도일 : 2011-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2011-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.9
누적 매도일 : 2012-01-03 00:00:00, 매도 종목수 : 1, 수익율 : 1.01
누적 매도일 : 2012-02-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.11
누적 매도일 : 2012-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.

(CODE              AAPL         AMZN        BND        GDX         GLD  \
 Date                                                                    
 2010-01-04   26.782711   133.899994  60.611969  44.908779  109.800003   
 2010-01-05   26.829010   134.690002  60.789135  45.341774  109.699997   
 2010-01-06   26.402260   132.250000  60.766037  46.443077  111.510002   
 2010-01-07   26.353460   130.000000  60.719822  46.217175  110.820000   
 2010-01-08   26.528664   133.520004  60.781410  46.913723  111.370003   
 ...                ...          ...        ...        ...         ...   
 2019-06-18  198.449997  1901.369995  82.397118  23.670000  127.120003   
 2019-06-19  197.869995  1908.790039  82.676468  24.000000  127.889999   
 2019-06-20  199.460007  1918.189941  82.806168  25.049999  131.110001   
 2019-06-21  198.779999  1911.300049  82.576698  25.209999  131.979996   
 2019-06-24  199.169998  1907.953857  82.726349  25.703501  133.501907   
 
 CODE               GM        MSFT  

In [154]:
import momentum2 as mmt2

In [157]:
import importlib

In [171]:
importlib.reload(mmt2)

<module 'momentum2' from 'c:\\Users\\moons\\Documents\\GitHub\\ubion-10\\python\\240929\\momentum2.py'>

In [163]:
# data_load({경로}) 
# 해당 경로에 있는 csv 파일을 모두 결합하여 
# (날짜,수정주가, 기준년월, 월별수익율, code값을 가진 데이터프레임), 
# (월말 데이터를 가지는 데이터프레임) 리턴 함수
stock_df, last_df = mmt2.data_load()

In [164]:
stock_df.head()

,Date,Adj Close,STD-YM,1m_rtn,CODE
7331,2010-01-04,26.782711,2010-01,0,AAPL
7332,2010-01-05,26.829010,2010-01,0,AAPL
7333,2010-01-06,26.402260,2010-01,0,AAPL
7334,2010-01-07,26.353460,2010-01,0,AAPL
7335,2010-01-08,26.528664,2010-01,0,AAPL


In [165]:
last_df.head()

,Date,CODE,1m_rtn
7349,2010-01-29,AAPL,0.897435
7368,2010-02-26,AAPL,1.050789
7391,2010-03-31,AAPL,1.124456
7412,2010-04-30,AAPL,1.106454
7432,2010-05-28,AAPL,0.964445


In [167]:
## create_position({월말 데이터프레임}, {상위 퍼센트})
## 리턴 값 : 구매 시기를 나타내는 딕셔너리 , 종목들의 리스트 
sig_dict, stock_codes = mmt2.create_position(last_df)

In [170]:
stock_df.head()

,Date,Adj Close,STD-YM,1m_rtn,CODE
7331,2010-01-04,26.782711,2010-01,0,AAPL
7332,2010-01-05,26.829010,2010-01,0,AAPL
7333,2010-01-06,26.402260,2010-01,0,AAPL
7334,2010-01-07,26.353460,2010-01,0,AAPL
7335,2010-01-08,26.528664,2010-01,0,AAPL


In [173]:
# create_trade_book({stock_df}, {종목 리스트}, {구매 시기 딕셔너리})
# 리턴 데이터 : 구매시기를 체크하여 구매 포지션을 지정한 데이터프레임
book = mmt2.create_trade_book(stock_df, stock_codes, sig_dict)

In [177]:
# trading({book}, {종목 리스트})
# 구매 내역을 추가하여 데이터프레임 리턴
trade_book = mmt2.trading(book, stock_codes)

In [178]:
# multi_returns({trade_book}, {종목 리스트})
# 수익율 데이터를 추가하여 데이터프레임을 리턴
rtn_book = mmt2.multi_returns(trade_book, stock_codes)

매수일 : 2010-02-01 00:00:00, 종목코드 : BND, 매수가 : 61.280487
매수일 : 2010-03-01 00:00:00, 종목코드 : AAPL, 매수가 : 26.154476
매도일 : 2010-03-01 00:00:00, 종목코드 : BND, 매도가 : 61.585163, 수익율 : 1.0049718273289832
매도일 : 2010-06-01 00:00:00, 종목코드 : AAPL, 매도가 : 32.642082, 수익율 : 1.248049549912604
매수일 : 2010-06-01 00:00:00, 종목코드 : GLD, 매수가 : 119.910004
매수일 : 2010-07-01 00:00:00, 종목코드 : GDX, 매수가 : 46.706642
매도일 : 2010-07-01 00:00:00, 종목코드 : GLD, 매도가 : 117.040001, 수익율 : 0.976065358149767
매도일 : 2010-08-02 00:00:00, 종목코드 : GDX, 매도가 : 45.228813, 수익율 : 0.9683593395560315
매수일 : 2010-08-02 00:00:00, 종목코드 : USM, 매수가 : 40.492416
매수일 : 2010-09-01 00:00:00, 종목코드 : GDX, 매수가 : 49.746994
매도일 : 2010-09-01 00:00:00, 종목코드 : USM, 매도가 : 36.667038, 수익율 : 0.9055285315650219
매수일 : 2010-10-01 00:00:00, 종목코드 : AMZN, 매수가 : 153.710007
매도일 : 2010-10-01 00:00:00, 종목코드 : GDX, 매도가 : 53.276817, 수익율 : 1.0709555033616704
매도일 : 2010-11-01 00:00:00, 종목코드 : AMZN, 매도가 : 162.580002, 수익율 : 1.0577060347150984
매수일 : 2010-11-01 00:00:00, 종목코드 : SLV, 매수가

In [179]:
# multi_acc_returns({rtn_book}, {종목 리스트})
# 누적수익율을 계산하여 데이터프레임에 대입한 결과와 총 누적 수익율을 리턴
acc_rtn_book, acc_rtn = mmt2.multi_acc_returns(rtn_book, stock_codes)

누적 매도일 : 2010-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2010-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.25
누적 매도일 : 2010-07-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.98
누적 매도일 : 2010-08-02 00:00:00, 매도 종목수 : 1, 수익율 : 0.97
누적 매도일 : 2010-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.91
누적 매도일 : 2010-10-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.07
누적 매도일 : 2010-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-02-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.16
누적 매도일 : 2011-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.01
누적 매도일 : 2011-06-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-07-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.99
누적 매도일 : 2011-08-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.05
누적 매도일 : 2011-09-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.06
누적 매도일 : 2011-10-03 00:00:00, 매도 종목수 : 1, 수익율 : 0.9
누적 매도일 : 2011-11-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.0
누적 매도일 : 2011-12-01 00:00:00, 매도 종목수 : 1, 수익율 : 0.9
누적 매도일 : 2012-01-03 00:00:00, 매도 종목수 : 1, 수익율 : 1.01
누적 매도일 : 2012-02-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.11
누적 매도일 : 2012-03-01 00:00:00, 매도 종목수 : 1, 수익율 : 1.